julie.tsitron@parks.nyc.gov 2/20/2020

# Imports and Connections to DBs

In [12]:
pwd

'C:\\Projects\\Structures\\develop'

In [1]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer, FeatureSet

sys.path.append('../')
from IPM_Shared_Code.Python.geo_functions import read_geosql
from IPM_Shared_Code.Python.utils import get_config
from IPM_Shared_Code.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *

In [2]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [3]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [4]:
con_data_parks = pyodbc.connect('Driver={' + driver + '};Server=' + data_parks_server +
                     ';Database=IPMDB;Trusted_Connection=Yes;')
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                           ';Database=;Trusted_Connection=Yes;')

## Deal with Dates

### m/d/Y H:M:S format:

In [5]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [6]:
# structures_dev['COMMISSIONDATE']

In [7]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [8]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [9]:
# structures_dev.columns.values

# Delta Table from structuresdb

In [10]:
# SPATIAL DATASET:
sql_str_deltas = 'select * FROM [structuresdb].[dbo].[tbl_delta_structures]'
struct_deltas = read_geosql(sql_str_deltas,
                            con,
                            geom_raw='shape',
                            geom_col='geometry')

In [11]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [12]:
struct_deltas.head()

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry


In [13]:
multipolygons = []

for i, row in struct_deltas.iterrows(): 
    if type(row['geometry'])==shapely.geometry.multipolygon.MultiPolygon:
        multipolygons.append(row['SYSTEM'])

if len(multipolygons)!=0:
    ids = '\n'.join(multipolygons)
    mssg = 'SYSTEM ID(s) of building(s) that are MultiPolygons:\n'+ids
    send_email('mycontacts.txt','multipoly_mssg.txt', subject = 'multipolygons', e=mssg)
    
struct_deltas = struct_deltas[~struct_deltas['SYSTEM'].isin(multipolygons)]

## Write Delta Table to geojson file

In [14]:
today = datetime.now().strftime('%Y%m%d')
snapshot = r'C:\\Projects\\Structures\\delta_snapshots/' + today

In [15]:
if not struct_deltas.empty:
    if os.path.exists(snapshot):
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')
    else:
        os.makedirs(snapshot)
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')

## Read geojson file to geojson object 

In [16]:
if not struct_deltas.empty:
    with open(snapshot+'/deltas.geojson') as data:
        geojson_deltas = json.load(data)

## Create arcgis featureSet from geojson object

In [17]:
if not struct_deltas.empty:
    fromJSON_deltas = FeatureSet.from_geojson(geojson_deltas)

# Connect to published datasets via GIS object

In [3]:
gis = GIS(url=portal)

# Make Edits

## Structures

### run delta archive stored proc

In [ ]:
crsr = con.cursor()
sqal_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

### Connect to structures feature layer directly

In [4]:
strct_lyr_url = structures_url

In [5]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

In [9]:
structures_features

<FeatureSet> 2702 features

In [10]:
len(struct_deltas)

NameError: name 'struct_deltas' is not defined

### EDIT

In [ ]:
if not struct_deltas.empty:
    num_updates = len(fromJSON_deltas.features)

In [ ]:
if not struct_deltas.empty:
    try:
        update_result = lyr_structures.edit_features(
            updates=fromJSON_deltas.features)
    except:
        error_mssg = 'Updates were not pushed successfully to Structures. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(num_updates) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_structures_mssg.txt', subject = 'structures push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the Structure layer.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures: NO UPDATES to push', e=mssg)

In [6]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

## Geosupport

### connect to geosupport service

In [6]:
geosupport_tbl_url = geosupport_url

In [7]:
tbl_geosupport = FeatureLayer(geosupport_tbl_url)
geosupport = tbl_geosupport.query()

### Get latest BINs from structures layer

In [8]:
structures_valid_BINs = structures_features.sdf[
    (structures_features.sdf['BIN'] != 0) &
    (~pd.isnull(structures_features.sdf['BIN']))] 

In [9]:
bins = structures_valid_BINs[structures_valid_BINs['BIN'].astype(int) %
                             100000 != 0]['BIN'].astype(int).to_list()

In [10]:
bins = [str(bins[i]) for i in range(0, len(bins))]

### run master geosupport function

In [11]:
df = master_geosupport_func(bins)

### prepare latest geosupport data to be pushed to prod

In [13]:
#Rename all of the columns from Geosupport so that they map exactly to the schema in SQL Server
dff = (df.rename(columns = {'out_zip_code': 'Zip_Code',
                               'b7sc': 'B7SC',
                               'out_b10sc1': 'B10SC',
                               'out_sanborn_boro': 'Boro_Code',
                               'bin': 'BIN',
                               'out_preferred_lgc': 'LGC',
                               'type_meaning': 'Address_Type',
                               'high_address_number': 'High_House_Num',
                               'low_address_number': 'Low_House_Num',
                               'out_sos_ind': 'Street_Side',
                               'street_name': 'Norm_Street_Name',
                               'out_usps_city_name': 'USPS_City',
                               'out_TPAD_bin_status': 'TPAD_BIN_Status',
                               'out_co': 'City_Council',
                               'out_ad': 'NYS_Assembly',
                               'out_sd': 'NYS_Senate',
                               'out_cd': 'US_Congress',
                               'out_nta': 'NTA_Code',
                               'out_fire_bat': 'Fire_Battalion',
                               'out_fire_co': 'Fire_Co_Num',
                               'out_fire_co_str': 'Fire_Co_Type',
                               'out_fire_div': 'Fire_Division',
                               'out_hurricane_zone': 'HEZ',
                               'out_police_patrol_boro': 'Police_Boro',
                               'Police Patrol Borough Command': 'Police_Boro_Com',
                               'out_police_pct': 'Police_Precinct',
                               'Sanitation Collection Scheduling Section and Subsection': 'Sanitation_Subsect',
                               'Sanitation District': 'Sanitation_District',
                               'Sanitation Recycling Collection Schedule': 'Sanitation_Recycling',
                               'out_san_reg': 'Sanitation_Reg_Pickup',
                               'out_physical_id': 'Physical_ID'})
       .reindex(columns = ['BIN', 'Boro_Code', 'Address_Type', 'Low_House_Num', 'High_House_Num', 'Norm_Street_Name', 'USPS_City', 
                           'Zip_Code', 'Physical_ID', 'B7SC', 'B10SC', 'LGC', 'Street_Side', 'TPAD_BIN_Status', 'HEZ', 
                           'Community_Board', 'City_Council', 'NYS_Assembly', 'NYS_Senate', 'US_Congress', 'NTA_Code', 
                           'Fire_Battalion', 'Fire_Co_Num', 'Fire_Co_Type', 'Fire_Division', 
                           'Police_Boro', 'Police_Boro_Com', 'Police_Precinct', 
                           'Sanitation_Subsect', 'Sanitation_District', 'Sanitation_Recycling', 'Sanitation_Reg_Pickup'])
       .drop_duplicates())

In [14]:
len(dff)

3540

In [15]:
dff.head(10)

,BIN,Boro_Code,Address_Type,Low_House_Num,High_House_Num,Norm_Street_Name,USPS_City,Zip_Code,Physical_ID,B7SC,...,Fire_Co_Num,Fire_Co_Type,Fire_Division,Police_Boro,Police_Boro_Com,Police_Precinct,Sanitation_Subsect,Sanitation_District,Sanitation_Recycling,Sanitation_Reg_Pickup
0,4592110,4,Non-Addressable Unnamed Building,None,None,BEACH 104 STREET,NaN,NaN,NaN,43361001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4453939,4,Ordinary Address Range,132A,132B,SERGEANT BEERS LANE,NaN,NaN,NaN,46189701,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4539845,4,Ordinary Address Range,340,340,STORY AVENUE,BAYSIDE,11359,0103875,46449501,...,Engine 306,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None
3,4539831,4,Ordinary Address Range,411A,411B,SHORE ROAD,NaN,NaN,NaN,46209501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4539891,4,Ordinary Address Range,615,615,LITTLE BAY ROAD,BAYSIDE,11359,0103838,45410701,...,Engine 306,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None
5,4539854,4,Ordinary Address Range,620,620,CIRCLE DRIVE,BAYSIDE,11359,0165595,43902001,...,Engine 306,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None
6,4539910,4,Ordinary Address Range,417,417,WALTER REED ROAD,BAYSIDE,11359,0103869,46676001,...,Engine 306,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None
7,4539838,4,Ordinary Address Range,405,405,SYLVESTER LANE,BAYSIDE,11359,0103866,46504001,...,Engine 306,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None
8,4539848,4,Ordinary Address Range,319,319,LITTLE BAY ROAD,None,None,None,45410701,...,None,None,None,None,NaN,None,NaN,NaN,NaN,None
9,4539846,4,Ordinary Address Range,403,403,SYLVESTER LANE,BAYSIDE,11359,0103866,46504001,...,Engine 306,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None


In [16]:
measurer = np.vectorize(len)

In [17]:
# type_df = pd.DataFrame({'df_name': dff.columns.tolist(),
#                         'df_type': dff.dtypes.astype(str).tolist(),
#                         'df_len' : measurer(dff.values.astype(str)).max(axis=0)})

In [18]:
# type_df.head()

,df_name,df_type,df_len
0,BIN,object,7
1,Boro_Code,object,4
2,Address_Type,object,33
3,Low_House_Num,object,12
4,High_House_Num,object,12


In [19]:
con_str = "Driver={SQL Server};Server=" + server + ";Database=structuresdb;Trusted_Connection=Yes;"
sa_con = quote_plus(con_str)
engine = sqlalchemy.create_engine(
    "mssql+pyodbc:///?odbc_connect={}".format(sa_con))

In [20]:
if len(dff) > 0:
    dff.to_sql('tbl_geosupport_address',
               engine,
               schema='dbo',
               if_exists='replace',
               index=False)

In [ ]:
dff.Boro_Code.fillna(0,inplace=True)

In [ ]:
geosupport_FSET = FeatureSet.from_dataframe(dff)

In [ ]:
len(geosupport_FSET.features)

### EDIT

In [ ]:
if len(dff) > 0: #len(geosupport_FSET.features):
    try:
        ## TRUNCATE AND REPLACE
        tbl_geosupport.delete_features(where="objectid > 0")
        add_to_geosupport = tbl_geosupport.edit_features(adds = geosupport_FSET.features)
    except:
        error_mssg = 'Updates were not pushed successfully to the geosupport table. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(len(dff)) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_geosupport_mssg.txt', subject = 'geosupport push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the geosupport table.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport: NO UPDATES to push', e=mssg)